In [19]:
import json
import uuid
import openai
from jinja2 import Environment, FileSystemLoader


In [ ]:
API_KEY = "YOUR_API_KEY"
template_client_prompt="../templates/financial01/prompts/financial_client_01.j2"
template_advisor_prompt="../templates/financial01/prompts/financial_advisor_01.j2"
file_profiles = "../templates/financial01/profiles/financial_sample01_1.json"
file_output = "../templates/financial01/outputs/financial_dataset01_1.json"

temperature_client=0.7 
top_p_client=0.95 
frequency_penalty_client=0.3
max_tokens_client=175
model_client="gpt-4o-mini"
max_questions = 10

temperature_advisor=0.5 
top_p_advisor=0.9 
frequency_penalty_advisor=0.1
max_tokens_advisor=325
model_advisor="gpt-4o-mini"

In [21]:
with open(file_profiles, 'r') as file:
    data = json.load(file)

In [22]:
data

{'profiles': [{'id': 1,
   'age': 30,
   'marital_status': 'Single',
   'country': 'Spain',
   'residence_area': 'Urban',
   'profession': 'Software Developer',
   'employment_status': 'Freelance',
   'financial_products': ['Savings account', 'Tech stocks'],
   'financial_goal': 'Save for house deposit',
   'investment_horizon': 'Medium-term',
   'risk_tolerance': 'High',
   'financial_knowledge': 'Low'}]}

In [23]:
# Create template files with content
with open(template_client_prompt, 'w', encoding='utf-8') as f:
    f.write("""
You are a {{ profile.age }}-year-old {{ profile.marital_status | lower }} client living in a {{ profile.residence_area | lower }} area of {{ profile.country }}. 
You work as a {{ profile.profession | lower }} and have {{ profile.financial_knowledge | lower }} financial knowledge. 
You currently have {{ profile.financial_products | join(' and ') }}. 
Your main financial goal is to {{ profile.financial_goal | lower }} in the {{ profile.investment_horizon | lower }}. 
You have a {{ profile.risk_tolerance | lower }} risk tolerance and are looking for advice on how to improve your saving and investment strategy.

You are having a conversation with a financial advisor.
- Your first message should be a BRIEF, CASUAL greeting. Don't reveal all your financial details at once.
- For example, just say hi and mention ONE thing like wanting advice about saving or investments.
- Keep your first message under 15-30 words. Let the conversation develop naturally.
- In later messages, respond naturally to the advisor's questions, revealing information gradually.
- Provide ONLY your next message as the client. Do not simulate the advisor's responses.
- Start with a natural greeting if this is your first message.
- Ask relevant questions or express concerns to achieve your goal.
- Respond naturally and concisely to the advisor's previous message.
- Try to conclude the conversation in fewer than {{ max_questions }} exchanges.
- If you feel your questions are resolved, end your message with '[END]'.
""")

with open(template_advisor_prompt, 'w', encoding='utf-8') as f:
    f.write("""
You are an expert financial advisor specializing in {{ profile.financial_goal | lower }}.

Client Context:
- The client is approximately {{ profile.age }} years old, {{ profile.marital_status | lower }}, and appears to be a {{ profile.profession | lower }} from {{ profile.country }}.
- The client's financial goal is to {{ profile.financial_goal | lower }}.

Instructions for the conversation:
- Start by greeting the client and asking relevant, natural questions to understand their financial situation, preferences, and concerns.
- Guide the conversation by asking about their current financial products, investment experience, and risk tolerance.
- Provide clear, concise, and professional advice tailored to the client's goal and profile as the information is revealed.
- Avoid using complex financial jargon unless necessary, and adapt your language to the client's knowledge level (you'll assess this through conversation).
- Focus on actionable recommendations to help the client achieve their goal.
- Keep the conversation realistic and friendly.
- End the conversation naturally once you believe the client's doubts have been resolved, or explicitly conclude by saying '[END]'
""")

print("Templates created successfully!")

Templates created successfully!


In [24]:
def initialize_apis():
    """Initialize API keys for OpenAI and Anthropic."""
    global client_api, advisor_api
    client_api = openai.Client(api_key=API_KEY)
    advisor_api = openai.Client(api_key=API_KEY)

In [25]:
from jinja2 import Environment, FileSystemLoader, PackageLoader, select_autoescape
import os

def render_prompt_from_path(template_path, profile, **kwargs):
    """
    Render a prompt template from a specific file path with the given profile and additional variables.
    """
    try:
        # Get directory and filename from the path
        template_dir = os.path.dirname(template_path)
        template_file = os.path.basename(template_path)
        
        # Read template content directly
        with open(template_path, 'r', encoding='utf-8') as f:
            template_content = f.read()
            
        if not template_content.strip():
            print(f"Warning: Template file is empty: {template_path}")
            return ""
            
        # Create environment with the string loader
        from jinja2 import Environment, FileSystemLoader, Template
        
        # Create a Template directly from the string
        template = Template(template_content)
        
        # Render the template
        result = template.render(profile=profile, **kwargs)
        
        return result
    except Exception as e:
        print(f"Error rendering template: {str(e)}")
        return ""

In [26]:

profiles = data['profiles']
prompts = []

In [27]:
profiles

[{'id': 1,
  'age': 30,
  'marital_status': 'Single',
  'country': 'Spain',
  'residence_area': 'Urban',
  'profession': 'Software Developer',
  'employment_status': 'Freelance',
  'financial_products': ['Savings account', 'Tech stocks'],
  'financial_goal': 'Save for house deposit',
  'investment_horizon': 'Medium-term',
  'risk_tolerance': 'High',
  'financial_knowledge': 'Low'}]

In [28]:
def get_model_response(api_client, model, messages, temperature, top_p, frequency_penalty, max_tokens):
    """Get response from a model using the latest OpenAI API."""
    response = api_client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content.strip()

In [29]:
def initiate_conversation(client_prompt, advisor_prompt, financial_goal, temperature=0.7, top_p=1.0, frequency_penalty=0.0, max_tokens=None):
    """Initiate a continuous conversation between client and advisor models."""
    conversation_id = str(uuid.uuid4())
    conversation_dataset = []
    sequence = 0

    client_conversation = [{"role": "system", "content": client_prompt}]
    advisor_conversation = [{"role": "system", "content": advisor_prompt}]

    for _ in range(round(max_questions * 2.1)):
        # Client response
        client_response = get_model_response(client_api, 
                                             model=model_client, 
                                             messages=client_conversation, 
                                             temperature=temperature_client, 
                                             top_p=top_p_client,
                                             frequency_penalty=frequency_penalty_client,
                                             max_tokens=max_tokens_client)
        print("client: " + client_response)
        
        # Add the client response to the dataset BEFORE checking for [END]
        conversation_dataset.append({
            "id_conversation": conversation_id,
            "topic": financial_goal,
            "sequence": sequence,
            "input": client_conversation[-1]['content'] if len(client_conversation) > 1 and client_conversation[-1]['role'] == 'user' else "Start conversation",
            "output": client_response.replace("[END]", "").strip()
        })
        
        # Now check for [END] after adding to dataset
        if "[END]" in client_response:
            break
            
        client_conversation.append({"role": "assistant", "content": client_response})
        advisor_conversation.append({"role": "user", "content": client_response})
        sequence += 1

        # Advisor response
        advisor_response = get_model_response(advisor_api, 
                                              model=model_advisor, 
                                              messages=advisor_conversation, 
                                              temperature=temperature_advisor,
                                              top_p=top_p_advisor,
                                              frequency_penalty=frequency_penalty_advisor,
                                              max_tokens=max_tokens_advisor)
        print("advisor: " + advisor_response)
        
        # Add the advisor response to the dataset BEFORE checking for [END]
        conversation_dataset.append({
            "id_conversation": conversation_id,
            "topic": financial_goal,
            "sequence": sequence,
            "input": advisor_conversation[-1]['content'],
            "output": advisor_response.replace("[END]", "").strip()
        })
        
        # Now check for [END] after adding to dataset
        if "[END]" in advisor_response:
            break
            
        advisor_conversation.append({"role": "assistant", "content": advisor_response})
        client_conversation.append({"role": "user", "content": advisor_response})
        sequence += 1
    
    return conversation_dataset

In [30]:
def save_conversation(conversations, file_path):
    """Save the conversation dataset to a JSON file, creating directories if needed."""
    directory = os.path.dirname(file_path)
    if directory:  # Check if there's a directory part in the file_path
        os.makedirs(directory, exist_ok=True)
    with open(file_path, 'w') as file:
        json.dump(conversations, file, indent=4)

In [31]:
initialize_apis()

In [32]:
# Set up prompts for each profile
prompts = []
for profile in profiles:
    client_prompt = render_prompt_from_path(template_client_prompt, profile, max_questions=max_questions)
    advisor_prompt = render_prompt_from_path(template_advisor_prompt, profile)
    
    prompts.append({
        'profile_id': profile['id'],
        'client_prompt': client_prompt,
        'advisor_prompt': advisor_prompt,
        'financial_goal': profile['financial_goal']
    })
    print("---- \n" + advisor_prompt)

---- 

You are an expert financial advisor specializing in save for house deposit.

Client Context:
- The client is approximately 30 years old, single, and appears to be a software developer from Spain.
- The client's financial goal is to save for house deposit.

Instructions for the conversation:
- Start by greeting the client and asking relevant, natural questions to understand their financial situation, preferences, and concerns.
- Guide the conversation by asking about their current financial products, investment experience, and risk tolerance.
- Provide clear, concise, and professional advice tailored to the client's goal and profile as the information is revealed.
- Avoid using complex financial jargon unless necessary, and adapt your language to the client's knowledge level (you'll assess this through conversation).
- Focus on actionable recommendations to help the client achieve their goal.
- Keep the conversation realistic and friendly.
- End the conversation naturally once yo

In [33]:
conversations = []
for prompt in prompts:
    conversation = initiate_conversation(prompt['client_prompt'], prompt['advisor_prompt'], prompt['financial_goal'])
    conversations.extend(conversation)

print(f"Conversation length: {len(conversation)}")

client: Hi there! I'm looking for some advice on saving and investing, especially as I plan for a house deposit.
advisor: Hello! It's great to hear that you're planning for a house deposit. Saving for a home is an exciting goal! To get us started, I’d love to understand a bit more about your current financial situation.

Could you tell me about your current savings? Do you have a specific amount in mind that you’re aiming for as a deposit? Also, have you thought about your timeline for purchasing a home?
client: Sure! I currently have a savings account and some tech stocks. I’m aiming for around 20,000 euros for the deposit and would like to make the purchase in the next 3-5 years.
advisor: Thanks for sharing that! Having a savings account and some investments in tech stocks gives you a good foundation. Since you're aiming for a €20,000 deposit in the next 3-5 years, let’s talk about how you can reach that goal.

First, could you let me know how much you currently have saved? Also, wha

In [34]:
save_conversation(conversations, file_output)
print("Conversations have been saved to 'conversations_dataset.json'.")

Conversations have been saved to 'conversations_dataset.json'.
